### Llama 2 13B Chat
#### RAG with LlamaIndex - Nvidia CUDA + WSL (Windows Subsystem for Linux) + Word documents + Local LLM

This notebook demonstrates the use of LlamaIndex for Retrieval Augmented Generation using Windows' WSL and an Nvidia's CUDA.

See the [README.md](README.md) file for help on how to run this.

#### 1. Prepare Llama Index for use

In [1]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, ServiceContext

/home/markwsl/miniconda3/envs/LlamaIndexRAGLinux/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### 2. Load the Word document(s)

Note: A fictitious story about Thundertooth a dinosaur who has travelled to the future. Thanks ChatGPT!

In [2]:
documents = SimpleDirectoryReader("./Data/").load_data()

#### 3. Instantiate the model

In [3]:
import torch

from llama_index.llms.llama_cpp import LlamaCPP
from llama_index.llms.llama_cpp.llama_utils import messages_to_prompt, completion_to_prompt
llm = LlamaCPP(
    model_url=None, # We'll load locally.
    model_path='./Models/llama-2-13b-chat.Q6_K.gguf',
    temperature=0.1,
    max_new_tokens=1024, # Increasing to support longer responses
    context_window=4096, # Llama 2 4096 context window
    generate_kwargs={},
    # set to at least 1 to use GPU
    model_kwargs={"n_gpu_layers": 43}, # 43 was a good amount of layers for the RTX 3090, you may need to decrease yours if you have less VRAM than 24GB
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    verbose=True
)

ggml_init_cublas: GGML_CUDA_FORCE_MMQ:   no
ggml_init_cublas: CUDA_USE_TENSOR_CORES: yes
ggml_init_cublas: found 1 CUDA devices:
  Device 0: NVIDIA GeForce RTX 3090, compute capability 8.6, VMM: yes
llama_model_loader: loaded meta data with 19 key-value pairs and 363 tensors from ./Models/llama-2-13b-chat.Q6_K.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 5120
llama_model_loader: - kv   4:                          llama.block_count u32              = 40
llama_model_loader: - kv   5:                  llama.feed_forward_length u32   

#### 4. Checkpoint

Are you running on GPU? The above output should include near the top something like:
> ggml_init_cublas: found 1 CUDA devices:

And in the full text near the bottom should be:
> llm_load_tensors: using CUDA for GPU acceleration

#### 5. Embeddings

Convert your source document text into embeddings.

The embedding model is from huggingface, this one performs well.

> https://huggingface.co/thenlper/gte-large


In [13]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="thenlper/gte-large", cache_folder=None)

#### 6. Prompt Template

Prompt template for Llama2 - see: https://huggingface.co/TheBloke/Llama-2-13B-chat-GPTQ/discussions/5#64b8e6cdf8bf823a61ed1243

The function creates a prompt given a dictionary of role + message entries (roles are assistant, which is the LLM, and user, which is your questions)

In [14]:
# Produces a prompt for the Llama2 model
def llama_v2_prompt(
    messages: list[dict]
):
    B_INST, E_INST = "[INST]", "[/INST]"
    B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
    DEFAULT_SYSTEM_PROMPT = f"""You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Please ensure that your responses are socially unbiased and positive in nature. If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information."""

    if messages[0]["role"] != "system":
        messages = [
            {
                "role": "system",
                "content": DEFAULT_SYSTEM_PROMPT,
            }
        ] + messages
    messages = [
        {
            "role": messages[1]["role"],
            "content": B_SYS + messages[0]["content"] + E_SYS + messages[1]["content"],
        }
    ] + messages[2:]

    messages_list = [
        f"{B_INST} {(prompt['content']).strip()} {E_INST} {(answer['content']).strip()} "
        for prompt, answer in zip(messages[::2], messages[1::2])
    ]
    messages_list.append(f"{B_INST} {(messages[-1]['content']).strip()} {E_INST}")

    return "".join(messages_list)

#### 7. Service Context

For chunking the document into tokens using the embedding model and our LLM

In [15]:
from llama_index.core import Settings

Settings.llm = llm
Settings.embed_model = embed_model
Settings.chunk_size=256 # Number of tokens in each chunk

#### 8. Index documents

In [16]:
index = VectorStoreIndex.from_documents(documents)

#### 9. Query Engine

Create a query engine, specifying how many citations we want to get back from the searched text (in this case 3).

The DB_DOC_ID_KEY is used to get back the filename of the original document

In [17]:
from llama_index.core.query_engine import CitationQueryEngine
query_engine = CitationQueryEngine.from_args(
    index,
    similarity_top_k=3,
    # here we can control how granular citation sources are, the default is 512
    citation_chunk_size=256,
)

# For citations we get the document info
DB_DOC_ID_KEY = "db_document_id"

#### 10. Prompt and Response function

Pass in a question, get a response back.

IMPORTANT: The prompt is set here, adjust it to match what you want the LLM to act like and do.

In [18]:
def RunQuestion(questionText):
    llama2Prompt = [
    {"role": "system", "content": "You are a technology specialist. Answer questions in a positive, helpful and empathetic way. If the answer is not in the following context return ONLY 'Sorry, I don't know the answer to that'."},
    {"role": "user", "content": f"{questionText}"}
]
    queryQuestion = llama_v2_prompt(llama2Prompt)

    response = query_engine.query(queryQuestion)

    return response

#### 11. Questions to test with

In [19]:
TestQuestions = [
    "Summarise the story for me",
    "Who was the main protagonist?",
    "Did they have any children? If so, what were their names?",
    "Did anything eventful happen?",
]

#### 12. Run Questions through model (this can take a while) and see citations

Runs each test question, saves it to a dictionary for output in the last step.

Note: Citations are the source documents used and the text the response is based on. This is important for RAG so you can reference these documents for the user, and to ensure it's utilising the right documents.

In [20]:
qa_pairs = []

for index, question in enumerate(TestQuestions, start=1):
    question = question.strip() # Clean up

    print(f"\n{index}/{len(TestQuestions)}: {question}")

    response = RunQuestion(question) # Query and get  response

    qa_pairs.append((question.strip(), str(response).strip())) # Add to our output array

    # Displays the citations
    for index, node in enumerate(response.source_nodes, start=1):
        print(f"{index}/{len(response.source_nodes)}: |{node.node.metadata['file_name']}| {node.node.get_text()}")

    # Uncomment the following line if you want to test just the first question
    # break 


1/4: Summarise the story for me


Llama.generate: prefix-match hit

llama_print_timings:        load time =     265.17 ms
llama_print_timings:      sample time =      37.69 ms /   142 runs   (    0.27 ms per token,  3767.28 tokens per second)
llama_print_timings: prompt eval time =     708.47 ms /  1085 tokens (    0.65 ms per token,  1531.48 tokens per second)
llama_print_timings:        eval time =    3134.61 ms /   141 runs   (   22.23 ms per token,    44.98 tokens per second)
llama_print_timings:       total time =    4156.39 ms /  1226 tokens
Llama.generate: prefix-match hit


1/4: |Thundertooth Part 1.docx| Source 1:
"Hello there, majestic creature. What brings you to our time?" Mayor Grace inquired, her voice calm and reassuring.



Thundertooth, though initially startled, found comfort in the mayor's soothing tone. In broken sentences, he explained his journey through time, the strange portal, and his hunger dilemma. Mayor Grace listened intently, her eyes widening with amazement at the tale of the prehistoric dinosaur navigating the future.



Realizing the dinosaur's predicament, Mayor Grace extended an invitation. "You are welcome in our city, Thundertooth. We can find a way to provide for you without causing harm to anyone. Let us work together to find a solution."



Grateful for the mayor's hospitality, Thundertooth followed her through the city. Together, they explored the futuristic marketplaces and innovative food labs, eventually discovering a sustainable solution that satisfied the dinosaur's hunger without compromising the well-being of the ci


llama_print_timings:        load time =     265.17 ms
llama_print_timings:      sample time =      42.63 ms /   156 runs   (    0.27 ms per token,  3659.22 tokens per second)
llama_print_timings: prompt eval time =     546.42 ms /   928 tokens (    0.59 ms per token,  1698.34 tokens per second)
llama_print_timings:        eval time =    3860.76 ms /   155 runs   (   24.91 ms per token,    40.15 tokens per second)
llama_print_timings:       total time =    4760.35 ms /  1083 tokens
Llama.generate: prefix-match hit


1/3: |Thundertooth Part 2.docx| Source 1:
Thundertooth



Embraced by the futuristic city and its inhabitants, Thundertooth found a sense of purpose beyond merely satisfying his hunger. Inspired by the advanced technology surrounding him, he decided to channel his creativity into something extraordinary. With the help of the city's brilliant engineers, Thundertooth founded a one-of-a-kind toy factory that produced amazing widgets – magical, interactive toys that captivated the hearts of both children and adults alike.



Thundertooth's toy factory became a sensation, and its creations were highly sought after. The widgets incorporated cutting-edge holographic displays, levitation technology, and even the ability to change shapes and colors with a mere thought. Children across the city rejoiced as they played with these incredible toys that seemed to bring their wildest fantasies to life.



As the years passed, Thundertooth's life took a heartwarming turn. He met a kind and intelligent


llama_print_timings:        load time =     265.17 ms
llama_print_timings:      sample time =      45.91 ms /   169 runs   (    0.27 ms per token,  3680.95 tokens per second)
llama_print_timings: prompt eval time =     582.34 ms /   973 tokens (    0.60 ms per token,  1670.85 tokens per second)
llama_print_timings:        eval time =    4099.73 ms /   168 runs   (   24.40 ms per token,    40.98 tokens per second)
llama_print_timings:       total time =    5071.85 ms /  1141 tokens
Llama.generate: prefix-match hit


1/4: |Thundertooth Part 2.docx| Source 1:
Thundertooth's toy factory became a sensation, and its creations were highly sought after. The widgets incorporated cutting-edge holographic displays, levitation technology, and even the ability to change shapes and colors with a mere thought. Children across the city rejoiced as they played with these incredible toys that seemed to bring their wildest fantasies to life.



As the years passed, Thundertooth's life took a heartwarming turn. He met a kind and intelligent dinosaur named Seraphina, and together they started a family. Thundertooth and Seraphina were blessed with four children, each with unique characteristics that mirrored the diversity of their modern world.



Lumina: The eldest of Thundertooth's children, Lumina inherited her mother's intelligence and her father's sense of wonder. With sparkling scales that emitted a soft glow, Lumina had the ability to generate light at will. She became fascinated with technology, often spending


llama_print_timings:        load time =     265.17 ms
llama_print_timings:      sample time =      52.24 ms /   198 runs   (    0.26 ms per token,  3790.20 tokens per second)
llama_print_timings: prompt eval time =     569.12 ms /   897 tokens (    0.63 ms per token,  1576.12 tokens per second)
llama_print_timings:        eval time =    3953.03 ms /   197 runs   (   20.07 ms per token,    49.84 tokens per second)
llama_print_timings:       total time =    4955.68 ms /  1094 tokens


1/3: |Thundertooth Part 2.docx| Source 1:
Thundertooth



Embraced by the futuristic city and its inhabitants, Thundertooth found a sense of purpose beyond merely satisfying his hunger. Inspired by the advanced technology surrounding him, he decided to channel his creativity into something extraordinary. With the help of the city's brilliant engineers, Thundertooth founded a one-of-a-kind toy factory that produced amazing widgets – magical, interactive toys that captivated the hearts of both children and adults alike.



Thundertooth's toy factory became a sensation, and its creations were highly sought after. The widgets incorporated cutting-edge holographic displays, levitation technology, and even the ability to change shapes and colors with a mere thought. Children across the city rejoiced as they played with these incredible toys that seemed to bring their wildest fantasies to life.



As the years passed, Thundertooth's life took a heartwarming turn. He met a kind and intelligent

#### 13. Output responses

In [21]:
for index, (question, answer) in enumerate(qa_pairs, start=1):
    print(f"{index}/{len(qa_pairs)} {question}\n\n{answer}\n\n--------\n")

1/4 Summarise the story for me

Of course! Here's a summary of the story:

In the future, a kind and welcoming mayor named Grace befriends a lost prehistoric dinosaur named Thundertooth who has traveled through time to their city. Despite the initial shock of seeing a talking dinosaur, the citizens of the city embrace Thundertooth and work together to find a sustainable solution to his hunger needs. Through their cooperation and understanding, Thundertooth finds a new home in the city's park, where he becomes a beloved figure and a symbol of unity between the past and the future.

Source: [1]

--------

2/4 Who was the main protagonist?

Based on the provided sources, the main protagonist of the story is Thundertooth, a talking dinosaur who traveled through time to the future and found a new purpose in life by founding a toy factory that produced amazing widgets. Source 1 describes Thundertooth's journey through time and his decision to channel his creativity into something extraordina